<a href="https://colab.research.google.com/github/iitdamansharma/Face-Detection/blob/main/Face_recuesion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Adding more Model to get better classification you know **

In [ ]:
!pip install opencv-python numpy scikit-learn joblib
import cv2, os, numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib


In [ ]:
from skimage.feature import hog

def extract_hog_features(img):
    img = cv2.resize(img, (80, 80))  # standardize size
    features, _ = hog(img,
                      orientations=9,
                      pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2),
                      visualize=True,
                      block_norm='L2-Hys')
    return features


In [ ]:
DATA_DIR = "data"
haar = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

X, y, id_map, label_map = [], [], {}, {}
cur_id = 0

for person in os.listdir(DATA_DIR):
    person_path = os.path.join(DATA_DIR, person)
    if not os.path.isdir(person_path): continue

    id_map[cur_id] = person
    label_map[person] = cur_id

    for img_name in os.listdir(person_path):
        path = os.path.join(person_path, img_name)
        img = cv2.imread(path)
        if img is None: continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = haar.detectMultiScale(gray, 1.3, 5)

        for (x, y1, w, h) in faces:
            face = gray[y1:y1+h, x:x+w]
            feat = extract_hog_features(face)
            X.append(feat)
            y.append(cur_id)
    cur_id += 1

X = np.array(X)
y = np.array(y)
print(f"✅ Loaded {len(X)} samples from {len(id_map)} people.")


✅ Loaded 25 samples from 4 people.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

best_acc = 0
best_k = 1
for k in range(1, 10):
    if len(X_train) < k: # Skip if training set is too small for current k
        continue
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
    knn.fit(X_train, y_train)
    acc = accuracy_score(y_test, knn.predict(X_test))
    if acc > best_acc:
        best_acc = acc
        best_k = k
print(f"🔥 Best K = {best_k}, Accuracy = {best_acc*100:.2f}%")

knn = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
knn.fit(X_train, y_train)

os.makedirs("model", exist_ok=True)
joblib.dump((knn, scaler, id_map), "model/face_knn.pkl")
print("✅ Model trained and saved successfully.")

🔥 Best K = 1, Accuracy = 80.00%
✅ Model trained and saved successfully.


In [ ]:
import cv2, numpy as np, os
from PIL import Image, ImageEnhance
import random

# Input image
# Make sure 'face.jpg' is in the same directory as your notebook, or provide the full path.
img = cv2.imread("/content/data/Himanshu/WhatsApp Image 2025-11-06 at 16.14.48_78310902.jpg")
os.makedirs("data/Himanshu", exist_ok=True)

# Check if the image was loaded successfully
if img is None:
    print("Error: Could not load the image 'face.jpg'. Please check the file path.")
else:
    def augment_image(img):
        pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        # Random brightness/contrast
        enhancer = ImageEnhance.Brightness(pil_img)
        pil_img = enhancer.enhance(random.uniform(0.6, 1.4))
        enhancer = ImageEnhance.Contrast(pil_img)
        pil_img = enhancer.enhance(random.uniform(0.7, 1.3))

        # Random rotation
        angle = random.randint(-15, 15)
        pil_img = pil_img.rotate(angle)

        # Convert back to numpy
        img_aug = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

        # Random flip
        if random.choice([True, False]):
            img_aug = cv2.flip(img_aug, 1)

        # Random noise
        noise = np.random.randint(0, 15, img_aug.shape, dtype='uint8')
        img_aug = cv2.add(img_aug, noise)

        return img_aug

    for i in range(15):
        aug_img = augment_image(img)
        cv2.imwrite(f"data/Himanshu/{i}.jpg", aug_img)

    print("✅ Generated 15 augmented images in Himanshu/aman/")

✅ Generated 15 augmented images in Himanshu/aman/
